In [ ]:
!pip install torch torchvision torchaudio
!pip install transformers
!pip install spacy
!pip install sentence-transformers
!pip install textblob
!pip install matplotlib

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
pip install language-tool-python


In [ ]:
pip install textblob


In [ ]:
# Imports for RoBERTa and PyTorch
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Imports for Preprocessing and Grammar
import spacy
from textblob import TextBlob

# Imports for Sentence Similarity and Embeddings
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Imports for Visualization
import matplotlib.pyplot as plt

# Utility imports
import numpy as np


In [8]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Define the model architecture (ensure it matches the saved model)
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=1)  # Adjust num_labels if needed

weights_path="/content/roberta_regression_model_weights.pt"

state_dict = torch.load(weights_path, map_location=torch.device('cpu'))

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-8-8553d93d3ad3>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlis

In [9]:
import torch
from transformers import RobertaModel, RobertaConfig

class CustomRobertaForRegression(torch.nn.Module):
    def __init__(self, model_name):
        super(CustomRobertaForRegression, self).__init__()
        self.roberta = RobertaModel.from_pretrained(model_name)
        self.regressor = torch.nn.Linear(self.roberta.config.hidden_size, 1)  # Adjust for regression

    def forward(self, input_ids, attention_mask=None):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        # Use the [CLS] token output for regression
        cls_output = outputs.last_hidden_state[:, 0, :]
        return self.regressor(cls_output)

# Load the custom model
model = CustomRobertaForRegression("roberta-base")
model.load_state_dict(torch.load("/content/roberta_regression_model_weights.pt"))
model.eval()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-9-5e31ce4bcb84>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recomm

CustomRobertaForRegression(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (

In [10]:
import language_tool_python
from textblob import TextBlob
import gradio as gr
from transformers import RobertaTokenizer, RobertaModel
import torch
import spacy
from sentence_transformers import SentenceTransformer, util
import numpy as np
from typing import Dict, List, Union
import re
from collections import Counter

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class RobertaRegressionModel(torch.nn.Module):
    def __init__(self, model_name='roberta-base'):
        super().__init__()
        self.roberta = RobertaModel.from_pretrained(model_name)
        self.regressor = torch.nn.Linear(self.roberta.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask=None):
        cls_output = self.roberta(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        return self.regressor(cls_output)

class EssayAnalyzer:
    def __init__(self):
        self.nlp = spacy.load('en_core_web_sm')
        self.sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.language_tool = language_tool_python.LanguageTool('en-US')

    def analyze_grammar(self, essay_text: str) -> Dict[str, Union[str, float]]:
        matches = self.language_tool.check(essay_text)
        word_count = len(essay_text.split())
        error_density = len(matches) / word_count if word_count > 0 else 0

        # Normalize grammar score (10 - error density * 100), bounded between 1-10
        grammar_score = max(1, min(10, 10 - (error_density * 100)))

        common_errors = Counter(match.ruleId for match in matches)

        return {
            'score': grammar_score,
            'error_count': len(matches),
            'error_density': error_density,
            'common_issues': dict(common_errors.most_common(3)),
            'feedback': self._generate_grammar_feedback(matches, error_density)
        }

    def analyze_coherence(self, essay_text: str) -> Dict[str, Union[str, float]]:
        doc = self.nlp(essay_text)
        sentences = list(doc.sents)

        # Analyze sentence length variation
        sent_lengths = [len(sent.text.split()) for sent in sentences]
        length_variance = np.var(sent_lengths) if sent_lengths else 0

        # Analyze transition words
        transition_words = set(['however', 'therefore', 'furthermore', 'moreover', 'consequently'])
        transition_count = sum(1 for sent in sentences if any(word.lower_ in transition_words for word in sent))

        # Calculate coherence score (1-10 scale)
        base_score = 5
        base_score += min(2.5, len(sentences) / 10)  # Reward reasonable length
        base_score += min(2.5, transition_count / len(sentences) * 5)  # Reward transitions
        coherence_score = max(1, min(10, base_score))

        return {
            'score': coherence_score,
            'sentence_count': len(sentences),
            'avg_sentence_length': np.mean(sent_lengths) if sent_lengths else 0,
            'transition_word_count': transition_count,
            'feedback': self._generate_coherence_feedback(len(sentences), transition_count)
        }

    def analyze_topic_relevance(self, essay_text: str, prompt_keywords: str) -> Dict[str, Union[str, float]]:
        sentences = [sent.strip() for sent in re.split('[.!?]', essay_text) if sent.strip()]

        # Calculate similarity scores
        embeddings = self.sentence_model.encode(sentences, convert_to_tensor=True)
        prompt_embedding = self.sentence_model.encode(prompt_keywords, convert_to_tensor=True)
        similarities = util.pytorch_cos_sim(prompt_embedding, embeddings)

        # Calculate average similarity and normalize to 1-10 scale
        avg_similarity = float(similarities.mean())
        relevance_score = max(1, min(10, avg_similarity * 10))

        return {
            'score': relevance_score,
            'avg_similarity': avg_similarity,
            'most_relevant_sentence': sentences[similarities.argmax().item()],
            'feedback': self._generate_relevance_feedback(avg_similarity)
        }

    def analyze_sentiment(self, essay_text: str) -> Dict[str, Union[str, float]]:
        blob = TextBlob(essay_text)
        polarity = blob.sentiment.polarity
        subjectivity = blob.sentiment.subjectivity

        # Convert polarity to 1-10 scale
        sentiment_score = max(1, min(10, (polarity + 1) * 5))

        return {
            'score': sentiment_score,
            'polarity': polarity,
            'subjectivity': subjectivity,
            'feedback': self._generate_sentiment_feedback(polarity, subjectivity)
        }

    def _generate_grammar_feedback(self, matches, error_density) -> str:
        if error_density == 0:
            return "Excellent grammar and spelling throughout the essay."
        elif error_density < 0.05:
            return f"Generally good grammar with {len(matches)} minor issues to review."
        else:
            return f"Consider reviewing the {len(matches)} grammar and spelling issues for improvement."

    def _generate_coherence_feedback(self, sentence_count, transition_count) -> str:
        if sentence_count < 5:
            return "The essay would benefit from more detailed development and better structure."
        elif transition_count < sentence_count / 4:
            return "Consider adding more transition words to improve flow between ideas."
        else:
            return "Good essay structure with effective use of transitions."

    def _generate_relevance_feedback(self, avg_similarity) -> str:
        if avg_similarity < 0.3:
            return "The essay may need to focus more directly on the given topic."
        elif avg_similarity < 0.6:
            return "Moderate topic relevance. Consider strengthening connections to the main prompt."
        else:
            return "Strong topic relevance throughout the essay."

    def _generate_sentiment_feedback(self, polarity, subjectivity) -> str:
        tone = "positive" if polarity > 0 else "negative" if polarity < 0 else "neutral"
        style = "objective" if subjectivity < 0.3 else "balanced" if subjectivity < 0.7 else "subjective"
        return f"The essay maintains a {tone} tone with a {style} writing style."

class EssayScorePredictor:
    def __init__(self, model_path, tokenizer_name='roberta-base'):
        self.device = device
        self.model = RobertaRegressionModel().to(self.device)
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))
        self.model.eval()
        self.tokenizer = RobertaTokenizer.from_pretrained(tokenizer_name)

    def predict(self, essay_text: str, max_length: int = 512) -> float:
        inputs = self.tokenizer(
            essay_text,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = inputs['input_ids'].to(self.device)
        attention_mask = inputs['attention_mask'].to(self.device)

        with torch.no_grad():
            score = self.model(input_ids=input_ids, attention_mask=attention_mask).cpu().item()

        # Normalize score to 1-10 scale
        normalized_score = max(1, min(10, score * 2))  # Assuming original score is 0-5
        return round(normalized_score, 2)

def predict_and_feedback(prompt_keywords: str, essay_text: str) -> Dict:
    analyzer = EssayAnalyzer()

    # Get detailed analysis
    grammar_analysis = analyzer.analyze_grammar(essay_text)
    coherence_analysis = analyzer.analyze_coherence(essay_text)
    relevance_analysis = analyzer.analyze_topic_relevance(essay_text, prompt_keywords)
    sentiment_analysis = analyzer.analyze_sentiment(essay_text)

    # Calculate overall score (weighted average)
    weights = {
        'grammar': 0.3,
        'coherence': 0.3,
        'relevance': 0.3,
        'sentiment': 0.1
    }

    overall_score = sum([
        grammar_analysis['score'] * weights['grammar'],
        coherence_analysis['score'] * weights['coherence'],
        relevance_analysis['score'] * weights['relevance'],
        sentiment_analysis['score'] * weights['sentiment']
    ])

    return {
        'Overall Score': round(overall_score, 2),
        'Grammar': grammar_analysis,
        'Coherence': coherence_analysis,
        'Topic Relevance': relevance_analysis,
        'Sentiment': sentiment_analysis
    }

# Gradio interface
def create_interface(model_path: str):
    inputs = [
        gr.Textbox(label="Essay Prompt Keywords", placeholder="Enter the essay prompt or topic keywords here..."),
        gr.Textbox(label="Essay Text", placeholder="Paste the essay text here...", lines=10),
    ]

    outputs = [
        gr.Number(label="Overall Score (1-10)"),
        gr.JSON(label="Detailed Analysis")
    ]

    def interface_function(prompt_keywords: str, essay_text: str):
        results = predict_and_feedback(prompt_keywords, essay_text)
        return results['Overall Score'], results

    demo = gr.Interface(
        fn=interface_function,
        inputs=inputs,
        outputs=outputs,
        title="Advanced Essay Scoring and Analysis",
        description="Get comprehensive feedback on your essay including grammar, coherence, topic relevance, and sentiment analysis."
    )

    return demo

if __name__ == "__main__":
    model_path = '/content/roberta_regression_model_weights.pt'
    demo = create_interface(model_path)
    demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9af72e267161489843.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
